In [83]:
#loading the packages
import pandas as pd
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import spacy
from collections import Counter
#import en_core_web_sm\
import re
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
import time
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.feature_extraction.text import CountVectorizer 
import pandas as pd
import json
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
import spacy
import langdetect
from langdetect import detect
import detectlanguage
from google_trans_new import google_translator  
import langid

In [85]:
#loading development datasets and concatenating thhem
dev_de = pd.read_json("dataset_de_dev.json", lines=True)
dev_es = pd.read_json("dataset_es_dev.json", lines=True)
dev_en = pd.read_json("dataset_en_dev.json", lines=True)
dev_fr = pd.read_json("dataset_fr_dev.json", lines=True)
dev= pd.concat([dev_fr, dev_es, dev_de, dev_en], axis=0)
dev.reset_index(drop=True, inplace=True)
dev

,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
0,fr_0112905,product_fr_0620743,reviewer_fr_0310864,1,Colis bien reçus avec le boîtier du jeu ouvert...,Commandé ailleurs si vous voulez des colis com...,fr,video_games
1,fr_0345365,product_fr_0942745,reviewer_fr_0244013,1,"Suite à une vérification, je vous informe que ...",COMMANDE NON RECUE,fr,office_product
2,fr_0474170,product_fr_0183830,reviewer_fr_0816879,1,"C'est un produit de piètre qualité sur quatre,...",Plastique sans aucune résistance !,fr,automotive
3,fr_0084156,product_fr_0195761,reviewer_fr_0933238,1,N'ont pas résistés. Piètre qualité,séparateur d'orteils,fr,sports
4,fr_0893613,product_fr_0077995,reviewer_fr_0492122,1,"Filament de qualité mais pas bien embobiné, ré...","ça coince, ça coince... que des problemes",fr,industrial_supplies
...,...,...,...,...,...,...,...,...
19995,en_0447642,product_en_0068982,reviewer_en_0061521,5,This ribbon is so adorable! Goes perfect with ...,So Cute!,en,home
19996,en_0950370,product_en_0563046,reviewer_en_0871798,5,I am in love with this kettle.,Perfect,en,kitchen
19997,en_0203466,product_en_0848682,reviewer_en_0474236,5,My Doberman Loves Having His Nails Trimmed and...,My Doberman Loves Them,en,pet_products
19998,en_0010627,product_en_0536493,reviewer_en_0546192,5,"I love my Fire.. I do everything on it, read, ...",Five Stars,en,other


In [96]:
#loading data pre-processed with unique linguistic characteristics
train=pd.read_excel('train_df_clean.xlsx')

In [86]:
#test set of Amazon Reviews Corpus
test=pd.read_excel('test.xlsx')

In [87]:
#Europarl test sample
formal_large=pd.read_excel('formal_large.xlsx')

In [90]:
#Europarl truncated
formal_short_nostop=pd.read_excel('formal_nostop.xlsx')
formal_short_nostop.head()

,Unnamed: 0,Unnamed: 0.1,txt,lang
0,0,0,"vigueur engagements Kosovo,",fr
1,1,1,cela n'implique d'énormes,fr
2,2,2,l'argent faveur pays,fr
3,3,3,"dépenses publiques, première",fr
4,4,4,année plus importante,fr


In [92]:
#TED Talks short text sample
functional=pd.read_excel('functional_words.xlsx')
functional=functional[functional['text'].str.split().str.len().ge(2)]
functional= functional[functional['text'].notnull()]

functional.reset_index(drop=True, inplace=True)

In [94]:
#TED Talks long text sample
ted_talk_full=pd.read_excel('tedtalk_full_large.xlsx')
ted_talk_full=ted_talk_full[ted_talk_full['text'].str.split().str.len().ge(4)]
ted_talk_full= ted_talk_full[ted_talk_full['text'].notnull()]
ted_talk_full.reset_index(drop=True, inplace=True)
ted_talk_full.head(2)

,Unnamed: 0,text,lang
0,0,I want you now to imagine a wearable robot tha...,en
1,1,We at Berkeley Bionics call these robots exosk...,en


In [ ]:
#training data with 
train_nostop=pd.read_excel('no_stopw.xlsx')
#cleaning the data set
train_nostop= train_nostop[train_nostop['review_body'].notnull()]
train_nostop.reset_index(drop=True, inplace=True)

In [ ]:
#training data with all stop words
train_sw=pd.read_excel('train_sw.xlsx')

In [ ]:
#traiing set with stemmed data
train_lemma=pd.read_excel('train_lemma.xlsx')

Checking the performance of logistic Regression trained over the data the pre-processed accouting for linguistic nuances

In [25]:
#feature representation
tvec_up_more = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=10000, ngram_range=(1, 3))
lr = LogisticRegression()
original_pipeline = Pipeline([
    ('vectorizer', tvec_up_more),
    ('classifier', lr)
])

In [27]:
#stratified cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = original_pipeline.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99997498 0.99984998 0.99892597 0.99984992]
recall:    [0.99944994 0.99979998 0.99992499 0.99942494]
f1 score:  [0.9997124  0.99982498 0.99942523 0.99963739]
--------------------------------------------------------
              de          en          es       fr
precision: [0.9998999  0.99965003 0.99857681 0.99992494]
recall:    [0.9990499  0.99982498 0.99994999 0.99922492]
f1 score:  [0.99947472 0.9997375  0.99926293 0.99957481]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99994996 0.99982499 0.9988262  0.99994998]
recall:    [0.99917492 0.99984998 0.99994999 0.99957496]
f1 score:  [0.99956229 0.99983749 0.99938778 0.99976243]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99994997 0.99984998 0.99910067 0.99992496]
recall:    [0.99944994 0.99984998 0.99994999 0.99957495]
f1

In [18]:
#training the model and testing on test and dev sets
train_X = train['new']   
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=10000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
lr = LogisticRegression()
lr.fit(X_train_tf, train_y)
y_pred = lr.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=lr.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99955000000000004956
[[4997    0    3    0]
 [   1 4999    0    0]
 [   0    0 5000    0]
 [   1    0    4 4995]]
------------------------------
accuracy validation set:  0.99965000000000003855
[[4996    2    2    0]
 [   0 4999    1    0]
 [   0    0 5000    0]
 [   0    1    1 4998]]


In [19]:
saving the model
import pickle
filename_lr = 'model_lr_10k.sav'
pickle.dump(lr, open(filename_lr, 'wb'))

In [20]:
# load the model from disk
loaded_model_train_lr = pickle.load(open(filename_lr, 'rb'))

In [21]:
#accuracy for long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_lr.score(x_formal, formal_large.lang)
print(result)

0.9974647601663117


In [22]:
#accuracy for short Europarl sample
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_lr.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.7897766368496268


In [23]:
#accuracy for long TED Talks
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=lr.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.98866757411986438964
[[29052    72   193    38]
 [   22 31950   122    62]
 [    6    69 30421   184]
 [   11    53   575 31327]]


In [24]:
#accuracy for short TED Talks
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=lr.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.91823472905012148981
[[22079   146  3020   101]
 [   90 25132  2119   742]
 [   10    41 24381   135]
 [   23   124  2131 25908]]


Random Forest 10,000 features

In [41]:
tvec = TfidfVectorizer(token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=10000, ngram_range=(1, 3))
#a pipeline for Random Forest with 10,000
rf = RandomForestClassifier()
rf_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', rf)
])


In [42]:
#stratified cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = rf_pipeline.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99987491 0.99984989 0.99760515 0.99987486]
recall:    [0.99922492 0.99924992 0.99984998 0.99887489]
f1 score:  [0.99954981 0.99954982 0.99872631 0.99937462]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99987487 0.99984992 0.99720691 0.99972464]
recall:    [0.9989749  0.99939994 0.99977498 0.99849985]
f1 score:  [0.99942468 0.99962488 0.99848929 0.99911187]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99979979 0.99964992 0.99772926 0.99982482]
recall:    [0.99884988 0.99952495 0.99969997 0.99892489]
f1 score:  [0.99932461 0.99958743 0.99871364 0.99937466]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99987489 0.9996249  0.99780406 0.99974971]
recall:    [0.99912491 0.99944994 0.99974997 0.99872484]
f1

In [31]:
#training the model and testing on test and dev sets
train_X = train['new']   
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=10000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
rf = RandomForestClassifier()
rf.fit(X_train_tf, train_y)
y_pred = rf.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=rf.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99909999999999998810
[[4996    0    4    0]
 [   2 4996    2    0]
 [   0    1 4998    1]
 [   0    1    7 4992]]
------------------------------
accuracy validation set:  0.99919999999999997708
[[4995    1    4    0]
 [   1 4998    1    0]
 [   0    0 4998    2]
 [   0    4    3 4993]]


In [32]:
#loading the model
import pickle
filename_rf = 'model_train_rf.sav'
pickle.dump(rf, open(filename_rf, 'wb'))

In [33]:
# load the model from disk
loaded_model_train_rf = pickle.load(open(filename_rf, 'rb'))


In [34]:
#accuracy for long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_rf.score(x_formal, formal_large.lang)
print(result)

0.9962731974444783


In [35]:
#accuracy for short Europarl sample
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_rf.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.6967263007463081


In [36]:
#accuracy for long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=rf.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.98113678648807556293
[[28825    64   400    66]
 [   28 31722   320    86]
 [   19    71 30326   264]
 [   15    74   935 30942]]


In [37]:
#accuracy for short TED Talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=rf.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.85109528922039512366
[[19986   109  5151   100]
 [  128 21847  5427   681]
 [   18    24 24379   146]
 [   35   159  3833 24159]]


Support Vector Machines 

In [ ]:
#pipeline one-vs-rest svm 
svc = OneVsRestClassifier(SVC())
svc_pipeline = Pipeline([
    ('vectorizer', tvec),
    ('classifier', svc)])


In [36]:
#stratified cross-validation
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = svc_pipeline.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99992499 0.99995    0.999975   0.999975  ]
recall:    [0.99992499 0.999975   0.999975   0.99994999]
f1 score:  [0.99992499 0.9999625  0.999975   0.9999625 ]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99997499 0.99967506 0.999975   0.999975  ]
recall:    [0.99972497 0.999975   0.999975   0.99992499]
f1 score:  [0.99984997 0.99982501 0.999975   0.99994999]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99994999 0.9999     1.         0.999975  ]
recall:    [0.99989999 0.999975   0.999975   0.999975  ]
f1 score:  [0.99992499 0.9999375  0.9999875  0.999975  ]
--------------------------------------------------------
              de          en          es       fr
precision: [0.999975 1.       0.999975 0.999975]
recall:    [1.       0.999975 0.999975 0.999975]
f1 score:  [0.9999

In [57]:
#training the model and testing on test and dev data sets
train_X = train['new']  
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=10000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
svc = OneVsRestClassifier(SVC())
svc.fit(X_train_tf, train_y)
y_pred = svc.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=svc.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99955000000000004956
[[4999    0    1    0]
 [   1 4999    0    0]
 [   1    0 4999    0]
 [   5    0    1 4994]]
------------------------------
accuracy validation set:  0.99965000000000003855
[[4997    3    0    0]
 [   0 5000    0    0]
 [   1    0 4999    0]
 [   1    2    0 4997]]


In [58]:
#loading into pickle
import pickle
filename1 = 'model__svc.sav'
pickle.dump(svc, open(filename1, 'wb'))

In [59]:
# load the model from disk
loaded_model_train_svc = pickle.load(open(filename1, 'rb'))

In [60]:
#accuracy for long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_svc.score(x_formal, formal_large.lang)
print(result)

0.9966661596187


In [61]:
#accuracy for short Europarl sample
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_svc.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.7933890859048325


In [62]:
#accuracy for long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=svc.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.97573233889349775261
[[29240    59    25    31]
 [   89 31982    44    41]
 [ 1887    75 28598   120]
 [  478    72    92 31324]]


In [63]:
#accuracy for short TED Talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=svc.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.91359175754836041694
[[24771   274   182   119]
 [ 1197 26097   410   379]
 [ 4187    73 20205   102]
 [ 1764   164   324 25934]]


Performance of MNB with data pre-processed with stop words removed

In [27]:
#stratified cross-validation 
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train_nostop.review_body
Y=train_nostop.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_up_more.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99994996 0.99957505 0.99935029 0.99982495]
recall:    [0.99932488 0.99979998 0.99992499 0.99964996]
f1 score:  [0.99963732 0.9996875  0.99963756 0.99973745]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99984991 0.99950007 0.99940018 0.99977492]
recall:    [0.99939989 0.99974997 0.99982498 0.99954994]
f1 score:  [0.99962485 0.99962501 0.99961253 0.99966242]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99979987 0.99945003 0.99974997 0.99925015]
recall:    [0.99932488 0.99962495 0.99972497 0.99957495]
f1 score:  [0.99956232 0.99953748 0.99973747 0.99941252]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99984994 0.99930024 0.99982496 0.99982495]
recall:    [0.99964993 0.99977497 0.99972497 0.99964996]
f1

In [272]:
#training the model on data with no stop words. Testing the model
train_X = train_nostop['review_body']   
train_y = train_nostop['language']  
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b', max_features=1000000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.97980000000000000426
[[4994    3    1    2]
 [ 247 4743    1    9]
 [  91    2 4902    5]
 [  26   12    5 4957]]
------------------------------
accuracy validation set:  0.98219999999999996199
[[4994    4    1    1]
 [ 221 4765    0   14]
 [  80    1 4913    6]
 [  11    9    8 4972]]


In [273]:
#saving the model
import pickle
filename2 = 'model_nostop_train_tfidf.sav'
pickle.dump(naive_bayes_classifier, open(filename2, 'wb'))

In [274]:
# load the model from disk
loaded_model_train_nostop = pickle.load(open(filename2, 'rb'))

In [275]:
#testing the model on long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_nostop.score(x_formal, formal_large.lang)
print(result)

0.8151936923232938


In [276]:
#testing the model on short Europarl
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_nostop.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9117662626369555


In [277]:
#testing the model on long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.72981789186272216785
[[24852  1893  1513  1097]
 [ 9520 19078   814  2744]
 [ 7655   914 21346   765]
 [ 3931  1081  1618 25336]]


In [39]:
#testing the model on short TED Talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.89526473413572926763
[[22366  1280  1209   491]
 [ 2695 23993   610   785]
 [  137   371 23654   405]
 [  578   931  1629 25048]]


MNB trained over the stemmed data

In [125]:
#stratified cross-validation  
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train_lemma.review_body
Y=train_lemma.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_up_more.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))

              de          en          es       fr
precision: [0.99987499 0.99989999 0.99989998 0.99984999]
recall:    [0.99989999 0.99992499 0.99982498 0.99987499]
f1 score:  [0.99988749 0.99991249 0.99986248 0.99986249]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99970001 0.99987498 0.999775   0.99984995]
recall:    [0.99982498 0.99984998 0.99987499 0.99964996]
f1 score:  [0.99976249 0.99986248 0.99982499 0.99974995]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99965007 0.99992499 0.9998     0.99992497]
recall:    [0.99994999 0.99984998 0.99989999 0.99959996]
f1 score:  [0.99980001 0.99988748 0.99984999 0.99976244]
--------------------------------------------------------
              de          en          es       fr
precision: [0.999825   0.99994999 0.99989999 0.99984997]
recall:    [0.99994999 0.99989999 0.99987499 0.99979997]
f1

In [282]:
#training the model over the stemmed data and testing it on test and dev sets.
train_X = train_lemma['review_body'] 
train_y = train_lemma['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b', max_features=1000000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99555000000000004601
[[4989    8    2    1]
 [   6 4989    1    4]
 [   7    8 4977    8]
 [  14    5   25 4956]]
------------------------------
accuracy validation set:  0.99550000000000005151
[[4984   11    4    1]
 [   3 4993    2    2]
 [  11    5 4976    8]
 [  14    4   25 4957]]


In [283]:
#loading the model
import pickle
filename7 = 'model_lemma_train_tfidf.sav'
pickle.dump(naive_bayes_classifier, open(filename7, 'wb'))

In [284]:
# load the model from disk
loaded_model_train_lemma = pickle.load(open(filename7, 'rb'))
#testing on long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_model_train_lemma.score(x_formal, formal_large.lang)
print(result)

#testing on short Europarl sample
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_lemma.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9956267112868877
0.920125
0.6798285079129837


In [285]:
#testing on long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.97109305153958291879
[[28978   283    55    39]
 [   49 32064    15    28]
 [  242   287 29244   907]
 [  244   118  1322 30282]]


In [83]:
#testing on short TED Talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.78453033470833100083
[[23996   878   226   246]
 [ 1310 25982   345   446]
 [12788  1378  9737   664]
 [ 3394   852   352 23588]]


Testing model's performance with no punctuation

In [97]:
import unicodedata 
import sys
#removing punctuation
punctuation = dict.fromkeys(i for i in range(sys.maxunicode)
                           if unicodedata.category(chr(i)).startswith('P'))
train['new'] = train['new'].apply(lambda item: ' '.join([string.translate(punctuation) for string in item.split()]))

In [98]:
#training MNB over the data with no punctuation
train_X = train['new']   
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
tf_vectorizer = TfidfVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b', max_features=1000000, ngram_range=(1, 3)) # or term frequency
X_train_tf = tf_vectorizer.fit_transform(train_X)
X_test_tf = tf_vectorizer.transform(test_X)
x_dev=tf_vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99995000000000000551
[[5000    0    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   1    0    0 4999]]
------------------------------
accuracy validation set:  0.99995000000000000551
[[4999    1    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   0    0    0 5000]]


In [99]:
#saving the model
import pickle
filename_punct = 'model_train_punct_removed.sav'
pickle.dump(naive_bayes_classifier, open(filename_punct, 'wb'))

In [100]:
# load the model from disk
loaded_punct_train = pickle.load(open(filename_punct, 'rb'))

In [101]:
#accuracy for long Europarl sample
x_formal=tf_vectorizer.transform(formal_large.txt)
result = loaded_punct_train.score(x_formal, formal_large.lang)
print(result)

0.9995056282324308


In [102]:
#accuracy for short Europarl sample
x_formal_short_nostop=tf_vectorizer.transform(formal_short_nostop.txt)
result = loaded_punct_train.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9758640766421426


In [103]:
#accuracy on long TED Talks sample
x_tedtalk=tf_vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.99551374469421782809
[[29248    87    10    10]
 [   16 32111     3    26]
 [  127   127 30390    36]
 [   27    76    12 31851]]


In [104]:
#accuracy on short TED talks sample
x_long_words=tf_vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.98978169557928841193
[[25023   231    28    64]
 [   59 27892    44    88]
 [  152   140 24214    61]
 [   54   145    19 27968]]


Testing the Use of CountVectorizer instead of TF-IDF

In [42]:
#stratified cross-validation
vectorizer = CountVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=1000000, ngram_range=(1, 3)) # or term frequency
clf = MultinomialNB()
bayes_pipeline_vec = Pipeline([
    ('vectorizer', vectorizer),
    ('classifier', clf)
])
#creating the function to print out the results of the modeling and pre   
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=777)
accuracy = []
precision = []
recall = []
f1 = []
X=train.new
Y=train.language
for train, test in kfold.split(X, Y):
    lr_fit = bayes_pipeline_vec.fit(X[train], Y[train])
    prediction = lr_fit.predict(X[test])
    scores = lr_fit.score(X[test],Y[test])
    #model_result=lr_fit.predict(testing)
    accuracy.append(scores * 100)
    precision.append(precision_score(Y[test], prediction, average='macro')*100)
    print('              de          en          es       fr')
    print('precision:',precision_score(Y[test], prediction, average=None))
    recall.append(recall_score(Y[test], prediction, average='macro')*100)
    print('recall:   ',recall_score(Y[test], prediction, average=None))
    f1.append(f1_score(Y[test], prediction, average='macro')*100)
    print('f1 score: ',f1_score(Y[test], prediction, average=None))
    print('-'*56)

print("accuracy: %.3f%% (+/- %.2f%%)" % (np.mean(accuracy), np.std(accuracy)))
print("precision: %.3f%% (+/- %.2f%%)" % (np.mean(precision), np.std(precision)))
print("recall: %.3f%% (+/- %.2f%%)" % (np.mean(recall), np.std(recall)))
print("f1 score: %.3f%% (+/- %.2f%%)" % (np.mean(f1), np.std(f1)))
#return(model_result)

              de          en          es       fr
precision: [0.99995  0.999975 1.       0.999975]
recall:    [0.999975   0.999975   1.         0.99994999]
f1 score:  [0.9999625 0.999975  1.        0.9999625]
--------------------------------------------------------
              de          en          es       fr
precision: [0.99987498 0.99980001 1.         0.999975  ]
recall:    [0.99984998 0.999975   0.99992499 0.99989999]
f1 score:  [0.99986248 0.9998875  0.99996249 0.99993749]
--------------------------------------------------------
              de          en          es       fr
precision: [0.999975 0.999925 1.       0.999975]
recall:    [0.99992499 0.999975   0.999975   1.        ]
f1 score:  [0.99994999 0.99995    0.9999875  0.9999875 ]
--------------------------------------------------------
              de          en          es       fr
precision: [0.999975 0.999975 0.999975 0.999975]
recall:    [0.999975   0.99994999 1.         0.999975  ]
f1 score:  [0.999975  0.999962

In [300]:
#training MNB using CountVectorizer instead of TF-IDF
train_X = train['new']   
train_y = train['language']   
test_X = test['review_body']
test_y = test['language']
vectorizer = CountVectorizer(stop_words=None,token_pattern='(?u)\\b\\w\\w*\\b|„|¿|¡', max_features=1000000, ngram_range=(1, 3)) # or term frequency
X_train_tf = vectorizer.fit_transform(train_X)
X_test_tf = vectorizer.transform(test_X)
x_dev=vectorizer.transform(dev.review_body)
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_y)
y_pred = naive_bayes_classifier.predict(X_test_tf)
score1 = metrics.accuracy_score(test_y, y_pred)
print("accuracy test set:   %0.20f" % score1)
print(metrics.confusion_matrix(test_y, y_pred))

print('------------------------------')

y_pred_2=naive_bayes_classifier.predict(x_dev)
score2 = metrics.accuracy_score(dev.language, y_pred_2)
print("accuracy validation set:  %0.20f" % score2)
print(metrics.confusion_matrix(dev.language, y_pred_2))

accuracy test set:   0.99995000000000000551
[[5000    0    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   1    0    0 4999]]
------------------------------
accuracy validation set:  0.99995000000000000551
[[4999    1    0    0]
 [   0 5000    0    0]
 [   0    0 5000    0]
 [   0    0    0 5000]]


In [302]:
#saving the model
import pickle
filename13 = 'model_count_train_count.sav'
pickle.dump(naive_bayes_classifier, open(filename13, 'wb'))

In [303]:
# load the model from disk
loaded_model_train_count = pickle.load(open(filename13, 'rb'))

In [304]:
#accuracu for log Europarl sample
x_formal=vectorizer.transform(formal_large.txt)
result = loaded_model_train_count.score(x_formal, formal_large.lang)
print(result)

0.9995563330291045


In [305]:
#accuracy for short EUroparl sample
x_formal_short_nostop=vectorizer.transform(formal_short_nostop.txt)
result = loaded_model_train_count.score(x_formal_short_nostop, formal_short_nostop.lang)
print(result)

0.9775313608214683


In [306]:
#accuracy for long TED Talks sample
x_tedtalk=vectorizer.transform(ted_talk_full.text)
print('------------------------------')

y_pred_5=naive_bayes_classifier.predict(x_tedtalk)
score5 = metrics.accuracy_score(ted_talk_full.lang, y_pred_5)
print("accuracy validation set:  %0.20f" % score5)
print(metrics.confusion_matrix(ted_talk_full.lang, y_pred_5))

------------------------------
accuracy validation set:  0.99618225311500763208
[[29284    56     6     9]
 [   11 32128     3    14]
 [  131   112 30402    35]
 [   26    59    12 31869]]


In [133]:
#accuracy for short TED Talks sample
x_long_words=vectorizer.transform(functional.text)
print('------------------------------')

y_pred_7=naive_bayes_classifier.predict(x_long_words)
score7 = metrics.accuracy_score(functional.lang, y_pred_7)
print("accuracy validation set:  %0.20f" % score7)
print(metrics.confusion_matrix(functional.lang, y_pred_7))

------------------------------
accuracy validation set:  0.99085532387787011999
[[25056   207    27    56]
 [   64 27921    27    71]
 [  147   126 24249    45]
 [   53   124    24 27985]]
